In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

In [2]:
input_text = "good morning"
target_text = "\tguten morgen\n"  # \t = start, \n = end for training convention

In [5]:
input_chars = sorted(list(set(input_text)))
target_chars = sorted(list(set(target_text)))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)

In [9]:
input_chars

[' ', 'd', 'g', 'i', 'm', 'n', 'o', 'r']

In [11]:
target_chars

['\t', '\n', ' ', 'e', 'g', 'm', 'n', 'o', 'r', 't', 'u']

In [13]:
max_encoder_seq_length = len(input_text)
max_decoder_seq_length = len(target_text)

input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [15]:
input_token_index

{' ': 0, 'd': 1, 'g': 2, 'i': 3, 'm': 4, 'n': 5, 'o': 6, 'r': 7}

In [17]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 'e': 3,
 'g': 4,
 'm': 5,
 'n': 6,
 'o': 7,
 'r': 8,
 't': 9,
 'u': 10}

In [19]:
reverse_target_char_index

{0: '\t',
 1: '\n',
 2: ' ',
 3: 'e',
 4: 'g',
 5: 'm',
 6: 'n',
 7: 'o',
 8: 'r',
 9: 't',
 10: 'u'}

In [21]:
encoder_input_data = np.zeros((1, max_encoder_seq_length), dtype='int32')
decoder_input_data = np.zeros((1, max_decoder_seq_length), dtype='int32')
decoder_target_data = np.zeros((1, max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [23]:
for t, char in enumerate(input_text):
    encoder_input_data[0, t] = input_token_index[char]

for t, char in enumerate(target_text):
    decoder_input_data[0, t] = target_token_index[char]
    if t > 0:
        decoder_target_data[0, t - 1, target_token_index[char]] = 1.0

# Define model
latent_dim = 256

In [25]:
encoder_input_data #good morning

array([[2, 6, 6, 1, 0, 4, 6, 7, 5, 3, 5, 2]])

In [29]:
decoder_target_data #one hot encoding of "guten morgen"

array([[[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [31]:
#encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(input_dim=num_encoder_tokens, output_dim=latent_dim)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb) 
#USING LSTM, you can use RNN, but for large sentences, you may face vanishing gradients
encoder_states = [state_h, state_c]

In [47]:
#decoder
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(input_dim=num_decoder_tokens, output_dim=latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [51]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=1, epochs=100, verbose=0)

In [53]:
encoder_model = Model(encoder_inputs, encoder_states)

In [55]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [57]:
dec_emb2 = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs)
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_states2 = [state_h2, state_c2]

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [59]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

# Prepare input for inference
test_input = np.zeros((1, max_encoder_seq_length), dtype='int32')
for t, char in enumerate(input_text):
    test_input[0, t] = input_token_index[char]

# Translate
translated = decode_sequence(test_input)
print(f"Input: {input_text}")
print(f"Output: {translated}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Input: good morning
Output: guten morgee
